<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. `sklearn.linear_model`. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите файл с ответами в соответствующем Programming Assignment. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk).

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [2]:
def write_answer_to_file(value, filename):
    with open(filename, 'w') as fout:
            fout.writelines([str(item) + ' ' for item in value])

In [3]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'data'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [4]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id', engine='c')

In [5]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id', engine='c')

In [6]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [7]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [8]:
from sklearn.preprocessing import LabelEncoder

In [94]:
y = train_df_400['user_id']
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [10]:
sites = ['site' + str(i) for i in range(1, 11)]

In [11]:
def to_csr(X):
    session_length = X.shape[1]
    data = [1] * X.ravel().shape[0]
    indices = X.ravel()
    indptr = range(0, X.ravel().shape[0] + session_length, session_length)
    return csr_matrix((data, indices, indptr))[:, 1:]

In [12]:
full_df = pd.concat([train_df_400.drop('user_id', axis=1).fillna(0), test_df_400.fillna(0)])
full_sparse = to_csr(full_df[sites].values)
split_idx = train_df_400.shape[0]
X_train_sparse = full_sparse[:split_idx, :]
X_test_sparse = full_sparse[split_idx:, :]
y = y.values

In [13]:
full_sparse

<229266x36656 sparse matrix of type '<class 'numpy.int64'>'
	with 2210802 stored elements in Compressed Sparse Row format>

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [14]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [15]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

In [16]:
y_valid

array([  7, 402, 798, ..., 632, 232, 635])

In [17]:
y_valid_for_vw

array([  4, 160, 312, ..., 254,  91, 256])

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [18]:
for row in train_df_part.values[:10]:
    print(' '.join(row.astype(int).astype('str')))

23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
8726 8725 665 8727 45 8725 45 5320 5320 5320
303 19 303 303 303 303 303 309 303 303
1359 925 1240 1360 1344 1359 1346 1345 1344 1345
11 85 52 85 11 52 11 85 10 85
83 1344 1240 1359 1345 85 1346 83 925 1359
13585 13585 13585 13585 13585 13585 13585 13585 13586 13585
29312 29311 29312 29312 -9223372036854775808 -9223372036854775808 -9223372036854775808 -9223372036854775808 -9223372036854775808 -9223372036854775808
21 62 18 1387 7 72 1386 1384 10 22
1563 1563 1563 1563 1563 1563 1563 1563 1563 1563


In [19]:
def to_vw_format(array, label=None):
    return str(label or '') + ' |text ' + ' '.join(document) + '\n'

In [63]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    with open(out_file, 'w') as fout:
        for i, row in enumerate(X):
            label = y[i] if not y is None else y
            string = str(label or '1') + ' | ' + ' '.join(row.astype(int).astype('str')) + '\n'
            fout.write(string)

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [64]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part.values, y_train_part_for_vw, out_file=os.path.join(PATH_TO_DATA, 'train_part.vw'))
arrays_to_vw(valid_df.values, y_valid_for_vw, out_file=os.path.join(PATH_TO_DATA, 'valid.vw'))
arrays_to_vw(train_df_400[sites].values, y_for_vw, out_file=os.path.join(PATH_TO_DATA, 'train.vw'))

CPU times: user 5.42 s, sys: 96.8 ms, total: 5.51 s
Wall time: 4.95 s


In [65]:
arrays_to_vw(test_df_400[sites].values,  out_file=os.path.join(PATH_TO_DATA, 'test.vw'))

In [66]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Результат должен получиться таким.**

In [67]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [68]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [69]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


In [27]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [28]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [29]:
!head -3 $PATH_TO_DATA/test.vw

 | 9 304 308 307 91 308 312 300 305 309
 | 838 504 68 11 838 11 838 886 27 305
 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [30]:
!vw --help 

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1

VW options:
  --ring_size arg                       size of example ring
  --onethread                           Disable parse thread

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial regressor(s)
  --initial_weight arg                  Set all weights t

In [31]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [80]:
%%time
!vw --oaa 400 -d $PATH_TO_DATA/train_part.vw \
-f 400users_part_model.vw --passes 3 -c -b 28 --random_seed 17

final_regressor = 400users_part_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = data/train_part.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976562 0.984375          128          128.0      348      346       11
0.941406 0.906250          256  

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [81]:
%%time
!vw -t -i 400users_part_model.vw -d $PATH_TO_DATA/valid.vw \
-p $PATH_TO_DATA/vw_valid_pred.csv --random_seed 17

only testing
predictions = data/vw_valid_pred.csv
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      228       11
0.656250 0.671875          256          256.0       14       14       11
0.646484 0.636

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [82]:
valid_preds = pd.read_csv('data/vw_valid_pred.csv', header=None)

In [83]:
np.unique(valid_preds)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(y_valid_for_vw, valid_preds.values)

0.34541741128414605

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1. Для `SGDClassifier` также укажите `max_iter=3`.**

In [37]:
logit = LogisticRegression(random_state=17, n_jobs=-1, solver='lbfgs', multi_class='auto')
sgd_logit = SGDClassifier(random_state=17, n_jobs=-1, max_iter=3)

In [38]:
%%time
logit.fit(X_train_part_sparse, y_train_part)
accuracy_score(y_valid, logit.predict(X_valid_sparse))

CPU times: user 1.03 s, sys: 2.54 s, total: 3.57 s
Wall time: 8min 58s


In [70]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)
accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse))

/Users/artem/miniconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


CPU times: user 24.6 s, sys: 1.65 s, total: 26.3 s
Wall time: 4.1 s


**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [73]:
sgd_valid_acc

0.278

In [72]:
vw_valid_acc = round(accuracy_score(y_valid_for_vw, valid_preds.values), 3)
sgd_valid_acc = round(accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse)), 3)
logit_valid_acc = round(accuracy_score(y_valid, logit.predict(X_valid_sparse)), 3)

In [41]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [42]:
write_answer_to_file(round(vw_valid_acc, 3), 'answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [74]:
%%time
!vw --oaa 400 -d $PATH_TO_DATA/train.vw \
-f 400users_model.vw --passes 3 -c -b 28 --random_seed 17

ignoring duplicate option: '-b 28'
final_regressor = 400users_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = data/train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976562 0.984375          128          128.0      348      346       11
0.94140

**Сделайте прогноз для тестовой выборки.**

In [84]:
%%time
!vw -t -i 400users_model.vw -d $PATH_TO_DATA/test.vw \
-p $PATH_TO_DATA/test_preds.csv --random_seed 17

only testing
predictions = data/test_preds.csv
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1       90       11
1.000000 1.000000            2            2.0        1       21       11
1.000000 1.000000            4            4.0        1      265       11
1.000000 1.000000            8            8.0        1      137       11
1.000000 1.000000           16           16.0        1      273       11
1.000000 1.000000           32           32.0        1      384       11
1.000000 1.000000           64           64.0        1      139       11
1.000000 1.000000          128          128.0        1       85       11
1.000000 1.000000          256          256.0        1       25       11
0.994141 0.988281 

In [85]:
preds = pd.read_csv('data/test_preds.csv', header=None).values.ravel()

In [86]:
preds = class_encoder.inverse_transform(preds - 1)

In [89]:
np.unique(preds)

array([  1,   6,   7,   8,   9,  10,  13,  16,  19,  25,  26,  27,  29,
        30,  34,  36,  39,  41,  42,  48,  49,  50,  54,  59,  60,  61,
        65,  67,  68,  70,  75,  76,  78,  83,  90,  94,  97,  99, 100,
       102, 104, 107, 111, 114, 115, 118, 120, 125, 126, 127, 130, 134,
       136, 138, 139, 143, 144, 149, 151, 156, 158, 160, 163, 166, 167,
       168, 171, 174, 176, 177, 179, 180, 183, 186, 188, 190, 191, 193,
       195, 197, 198, 200, 202, 205, 208, 213, 216, 217, 224, 232, 236,
       239, 245, 247, 251, 253, 255, 256, 262, 264, 266, 268, 269, 273,
       274, 280, 288, 289, 290, 291, 292, 294, 296, 298, 302, 305, 306,
       307, 308, 310, 314, 315, 317, 318, 319, 324, 325, 326, 330, 341,
       343, 347, 351, 352, 353, 354, 356, 357, 358, 360, 361, 363, 364,
       368, 370, 371, 373, 375, 381, 384, 387, 388, 389, 391, 392, 393,
       402, 404, 405, 406, 407, 408, 409, 413, 417, 425, 429, 432, 436,
       438, 439, 440, 441, 444, 445, 446, 448, 454, 455, 462, 46

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [47]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [95]:
vw_pred = pd.read_csv('data/test_preds.csv', header=None).values.ravel()
vw_pred = class_encoder.inverse_transform(vw_pred - 1)

In [96]:
np.unique(vw_pred)

array([  1,   6,   7,   8,   9,  10,  13,  16,  19,  25,  26,  27,  29,
        30,  34,  36,  39,  41,  42,  48,  49,  50,  54,  59,  60,  61,
        65,  67,  68,  70,  75,  76,  78,  83,  90,  94,  97,  99, 100,
       102, 104, 107, 111, 114, 115, 118, 120, 125, 126, 127, 130, 134,
       136, 138, 139, 143, 144, 149, 151, 156, 158, 160, 163, 166, 167,
       168, 171, 174, 176, 177, 179, 180, 183, 186, 188, 190, 191, 193,
       195, 197, 198, 200, 202, 205, 208, 213, 216, 217, 224, 232, 236,
       239, 245, 247, 251, 253, 255, 256, 262, 264, 266, 268, 269, 273,
       274, 280, 288, 289, 290, 291, 292, 294, 296, 298, 302, 305, 306,
       307, 308, 310, 314, 315, 317, 318, 319, 324, 325, 326, 330, 341,
       343, 347, 351, 352, 353, 354, 356, 357, 358, 360, 361, 363, 364,
       368, 370, 371, 373, 375, 381, 384, 387, 388, 389, 391, 392, 393,
       402, 404, 405, 406, 407, 408, 409, 413, 417, 425, 429, 432, 436,
       438, 439, 440, 441, 444, 445, 446, 448, 454, 455, 462, 46

In [97]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
%%time
logit.fit(X_train_sparse, y)

In [100]:
logit_test_pred = logit.predict(X_test_sparse)

In [101]:
%%time
sgd_logit.fit(X_train_sparse, y)
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

/Users/artem/miniconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


CPU times: user 37.3 s, sys: 1.98 s, total: 39.3 s
Wall time: 5.61 s


In [102]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


In [104]:
vw_lb_score, sgd_lb_score, logit_lb_score = 0.18768, 0.15153, 0.19422

write_answer_to_file(round(vw_lb_score, 3), 'answer6_4.txt')
write_answer_to_file(round(sgd_lb_score, 3), 'answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answer6_6.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**